In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile7 = pd.read_pickle("BBB_Train_Decile7.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile7.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile7, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 7')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3479 - acc: 0.8733 - val_loss: 0.3316 - val_acc: 0.8679
Epoch 39/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3474 - acc: 0.8733 - val_loss: 0.3247 - val_acc: 0.8705
Epoch 40/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3448 - acc: 0.8746 - val_loss: 0.3289 - val_acc: 0.8666
Epoch 41/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3451 - acc: 0.8737 - val_loss: 0.3314 - val_acc: 0.8653
Epoch 42/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3411 - acc: 0.8704 - val_loss: 0.3324 - val_acc: 0.8692
Epoch 43/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3455 - acc: 0.8714 - val_loss: 0.3265 - val_acc: 0.8692
Epoch 44/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3445 - acc: 0.8740 - val_loss: 0.3289 - val_acc: 0.8666

Epoch 00044: ReduceLROnPlateau reducing learning rate to 1e-

Epoch 98/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3341 - acc: 0.8753 - val_loss: 0.3229 - val_acc: 0.8679
Epoch 99/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3323 - acc: 0.8798 - val_loss: 0.3225 - val_acc: 0.8692
Epoch 100/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3340 - acc: 0.8733 - val_loss: 0.3222 - val_acc: 0.8692
Test accuracy: 0.8691709844559585
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5823 - acc: 0.6751 - val_loss: 0.4644 - val_acc: 0.7966
Epoch 2/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4198 - acc: 0.8393 - val_loss: 0.3458 - val_acc: 0.8679
Epoch 3/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3689 - acc: 0.8656 - val_loss: 0.3248 - val_acc: 0.8718
Epoch 4/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3516 - acc: 0.8678 - val_l

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3194 - acc: 0.8772 - val_loss: 0.3068 - val_acc: 0.8756
Epoch 57/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3172 - acc: 0.8788 - val_loss: 0.3067 - val_acc: 0.8744
Epoch 58/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3242 - acc: 0.8733 - val_loss: 0.3065 - val_acc: 0.8756
Epoch 59/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3154 - acc: 0.8811 - val_loss: 0.3067 - val_acc: 0.8769
Epoch 60/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3158 - acc: 0.8801 - val_loss: 0.3067 - val_acc: 0.8769
Epoch 61/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3145 - acc: 0.8811 - val_loss: 0.3068 - val_acc: 0.8756
Epoch 62/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3186 - acc: 0.8763 - val_loss: 0.3068 - val_acc: 0.8756
Epoch 63/150
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3120 - acc: 0.8798 - val_loss: 0.3067 - val_acc: 0.8756
Epoch 117/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3168 - acc: 0.8743 - val_loss: 0.3068 - val_acc: 0.8744
Epoch 118/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3155 - acc: 0.8792 - val_loss: 0.3066 - val_acc: 0.8756
Epoch 119/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3155 - acc: 0.8766 - val_loss: 0.3066 - val_acc: 0.8756
Epoch 120/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3159 - acc: 0.8801 - val_loss: 0.3068 - val_acc: 0.8744
Epoch 121/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3156 - acc: 0.8769 - val_loss: 0.3066 - val_acc: 0.8756
Epoch 122/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3189 - acc: 0.8776 - val_loss: 0.3065 - val_acc: 0.8756
Epoch 123/150
3087/3087 [==============================

...

Epoch 63/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3574 - acc: 0.8695 - val_loss: 0.3479 - val_acc: 0.8653
Epoch 64/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3537 - acc: 0.8695 - val_loss: 0.3479 - val_acc: 0.8653
Epoch 65/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3553 - acc: 0.8659 - val_loss: 0.3482 - val_acc: 0.8666
Epoch 66/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3545 - acc: 0.8707 - val_loss: 0.3482 - val_acc: 0.8666
Epoch 67/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3521 - acc: 0.8695 - val_loss: 0.3483 - val_acc: 0.8666
Epoch 68/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3541 - acc: 0.8685 - val_loss: 0.3472 - val_acc: 0.8653
Epoch 69/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3557 - acc: 0.8691 - val_loss: 0.3478 - val_acc: 0.8666
Epoch 70/100
3087/3087 [========================

3087/3087 [==============================] - 7s 2ms/step - loss: 0.3127 - acc: 0.8763 - val_loss: 0.3097 - val_acc: 0.8705
Epoch 23/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3172 - acc: 0.8763 - val_loss: 0.3065 - val_acc: 0.8756

Epoch 00023: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 24/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3150 - acc: 0.8827 - val_loss: 0.3067 - val_acc: 0.8756
Epoch 25/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3148 - acc: 0.8779 - val_loss: 0.3065 - val_acc: 0.8756
Epoch 26/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3127 - acc: 0.8795 - val_loss: 0.3067 - val_acc: 0.8756
Epoch 27/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3215 - acc: 0.8788 - val_loss: 0.3067 - val_acc: 0.8756
Epoch 28/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3155 - acc: 0.8779 - val_loss: 0.30

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3158 - acc: 0.8785 - val_loss: 0.3051 - val_acc: 0.8769
Epoch 82/200
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3151 - acc: 0.8818 - val_loss: 0.3051 - val_acc: 0.8769
Epoch 83/200
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3177 - acc: 0.8766 - val_loss: 0.3054 - val_acc: 0.8769
Epoch 84/200
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3164 - acc: 0.8795 - val_loss: 0.3053 - val_acc: 0.8769
Epoch 85/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3132 - acc: 0.8818 - val_loss: 0.3054 - val_acc: 0.8769
Epoch 86/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3130 - acc: 0.8779 - val_loss: 0.3055 - val_acc: 0.8756
Epoch 87/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3177 - acc: 0.8798 - val_loss: 0.3056 - val_acc: 0.8756
Epoch 88/200
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3175 - acc: 0.8814 - val_loss: 0.3048 - val_acc: 0.8769
Epoch 142/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3177 - acc: 0.8769 - val_loss: 0.3047 - val_acc: 0.8769
Epoch 143/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3135 - acc: 0.8782 - val_loss: 0.3050 - val_acc: 0.8756
Epoch 144/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3172 - acc: 0.8792 - val_loss: 0.3050 - val_acc: 0.8756
Epoch 145/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3146 - acc: 0.8795 - val_loss: 0.3051 - val_acc: 0.8769
Epoch 146/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3121 - acc: 0.8798 - val_loss: 0.3051 - val_acc: 0.8769
Epoch 147/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3152 - acc: 0.8805 - val_loss: 0.3051 - val_acc: 0.8769
Epoch 148/200
3087/3087 [==============================

Test accuracy: 0.8769430051813472
Train on 3087 samples, validate on 772 samples
Epoch 1/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.6194 - acc: 0.6553 - val_loss: 0.5729 - val_acc: 0.7176
Epoch 2/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5592 - acc: 0.7098 - val_loss: 0.4793 - val_acc: 0.7552
Epoch 3/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4378 - acc: 0.8319 - val_loss: 0.4229 - val_acc: 0.8433
Epoch 4/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3919 - acc: 0.8581 - val_loss: 0.3646 - val_acc: 0.8640
Epoch 5/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3693 - acc: 0.8620 - val_loss: 0.3434 - val_acc: 0.8640
Epoch 6/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3631 - acc: 0.8701 - val_loss: 0.3357 - val_acc: 0.8575
Epoch 7/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3447 - acc: 0.8750 - val_loss: 0.3337

In [ ]:
0.8297